In [ ]:
!pip install -q datasets

In [ ]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import spacy
from datasets import load_dataset
from typing import List, Tuple

https://huggingface.co/datasets/Davlan/sib200 - качаем датасет отсюда

In [ ]:
# функция для загрузки датасета
'''
Необходимо проверить, совпадают ли категории в тренировочной, валидационной и тестовой выборках. Если нет, выводить сообщение об ошибке и RuntimeError.
В переменную categories положить все классы, которые есть в тренировочной выборке. В валидационной и тестовой оставить только категории, которые есть в тренировочной.
Конфигурация rus_Cyrl.
'''
def load_sib200_ru():# -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    train_dataset = load_dataset("Davlan/sib200", 'rus_Cyrl', split = 'train')
    test_dataset = load_dataset("Davlan/sib200", 'rus_Cyrl', split = 'test')

    categories = list(set(train_dataset['category']))
    for category in set(test_dataset['category']):
      if category not in categories:
        index = test_dataset['category'].index(category)
        test_dataset[0].pop(index)
        test_dataset[1].pop(index)

        raise RuntimeError(f"Category {category} not found in train_dataset")

    X_train, y_train = train_dataset['text'], train_dataset['category']
    X_test, y_test = test_dataset['text'], test_dataset['category']

    return (X_train, y_train), (X_test, y_test), categories

In [ ]:
'''
Убрать знаки препинания, выполнить токенизацию и лемматизацию.
Вместо всех числовых токенов вставить <NUM>.
Привести к нижнему регистру.
'''
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    s = re.sub(r'[^\w\s]', '', s)

    doc = nlp_pipeline(s)

    lemmas = []
    for token in doc:
        if token.like_num:
            lemmas.append('<NUM>')
        else:
            lemmas.append(token.lemma_.lower())

    return ' '.join(lemmas)

In [ ]:
train_data, test_data, classes_list = load_sib200_ru()

In [ ]:
# напечатать список категорий
print(classes_list)

['entertainment', 'sports', 'geography', 'science/technology', 'politics', 'travel', 'health']


In [ ]:
# проверить размерности всех полученных выборок
print(f'Size of training data - {len(train_data[0])}')
print(f'Size of testing data - {len(test_data[0])}')

Size of training data - 701
Size of testing data - 204


In [ ]:
# загрузить пайплайн ru_core_news_sm
!python -m spacy download ru_core_news_sm
nlp = spacy.load('ru_core_news_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 73.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# применить normalize_text и nlp ко всем выборкам

for index in range(len(train_data[0])):
  train_data[0][index] = normalize_text(train_data[0][index], nlp)

for index in range(len(test_data[0])):
  test_data[0][index] = normalize_text(test_data[0][index], nlp)

In [ ]:
print(train_data[0][0])
print(test_data[0][15])

турция с <NUM> сторона окружить море на запад   эгейский на север   чёрный и на юг   средиземный
немецкий подводный лодка называться uboat немец очень хорошо разбираться в навигация и управление свой подлодка


In [ ]:
# создать Pipeline для векторизации текстов и логистической регрессии

classifier = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer()),
    ('cls', LogisticRegression(random_state=42))
])

In [ ]:
#подобрать параметры с помощью GridSearchCV

cv = GridSearchCV(
    estimator=classifier,
    param_grid={
        'vectorizer__ngram_range': [(1, 1), (1, 2)], #(1,1)
        'cls__C': [1e-1, 1, 10, 100, 1000], #1000
        'cls__penalty': ['l1', 'l2'], #l2
        'cls__solver': ['liblinear', 'saga'],  #liblinear
        'cls__max_iter': [100] #100
    },
    scoring='f1_macro',
    cv=5,
    refit=True,
    n_jobs=-1,
    verbose=True
)


In [ ]:
# обучить классификатор с перебором гиперпараметров
cv.fit(train_data[0], train_data[1])
best_params = cv.best_params_
best_model = cv.best_estimator_
best_f1_score = cv.best_score_


Fitting 5 folds for each of 40 candidates, totalling 200 fits


In [ ]:
# напечатать наилучшие параметры
best_params

{'cls__C': 1000,
 'cls__max_iter': 100,
 'cls__penalty': 'l2',
 'cls__solver': 'liblinear',
 'vectorizer__ngram_range': (1, 1)}

In [ ]:
# напечатать лучший F1
best_f1_score

0.6493254943710063

In [ ]:
# напечатать размер словаря векторизованных текстов

vectorizer = best_model.named_steps['vectorizer']
print(f'Length of dict - {len(vectorizer.get_feature_names_out())}')

Length of dict - 4338


In [ ]:
# получить предсказания на тестовой выборке и вывести classification_report
y_pred = best_model.predict(test_data[0])
print(classification_report(test_data[1], y_pred, target_names = classes_list))

                    precision    recall  f1-score   support

     entertainment       0.80      0.42      0.55        19
            sports       0.60      0.53      0.56        17
         geography       0.53      0.41      0.46        22
science/technology       0.71      0.80      0.75        30
          politics       0.67      0.80      0.73        51
            travel       0.80      0.80      0.80        25
            health       0.60      0.62      0.61        40

          accuracy                           0.67       204
         macro avg       0.67      0.63      0.64       204
      weighted avg       0.67      0.67      0.66       204



In [ ]:
print(f'travel - {train_data[1].count("travel")}')
print(f'geography - {train_data[1].count("geography")}')

travel - 138
geography - 58


In [ ]:
# проанализировать полученные результаты и написать выводы
"""Исходя из метрик, полученных на тестовой выборке, можно сделать вывод, что в датасете присутствует дисбаланс классов,
так как показатель ф1 достаточно сильно разнится(например для категорий география и путешествия показатели значительно отличаются).
И в целом показатель точности не очень впечатляющий, так как датасет маленький."""